# Import Libraries

In [59]:
#Preparing all libraries
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import golden
from scipy.optimize import fminbound
from scipy.linalg import svd
from scipy.stats import trim_mean
import addcopyfighandler
import statistics
import math
import time
from tqdm import trange
from matplotlib import cm

# Initialize Parameters

In [60]:
#### Grid Parameters
TE_array = np.arange(8, 512, 8) #ms units

#### NLLS parameters
lower_bound = (-1,-1,1,1)
upper_bound = (1,1,300,1000)


#### Model parameters
c1 = 0.5
c2 = 0.5
T21 = 45
T22 = 200
T11 = 600
T12 = 1200

true_params = np.array([T11, T12, c1, c2, T21, T22])

#### Nullpoint Values
TI1star = np.log(2)*T11
TI2star = np.log(2)*T12

#### Process parameters
SNR_fixed = 1000
num_iters = 10
rand_seed = 10
num_starts = 20

# Initialize Signal Functions

In [61]:
def S_biX_6p(TE, TI, T11, T12, c1, c2, T21, T22):
    exp1 = c1*(1-2*np.exp(-TI/T11))*np.exp(-TE/T21)
    exp2 = c2*(1-2*np.exp(-TI/T12))*np.exp(-TE/T22)
    return exp1 + exp2

def S_biX_4p(TE, d1, d2, T21, T22):
    exp1 = d1*np.exp(-TE/T21)
    exp2 = d2*np.exp(-TE/T22)
    return exp1 + exp2

#Defining the monoExp function of interest
def S_moX_2p(TE, d, T2):
    return d*np.exp(-TE/T2)

def add_noise(data,SNR):
    sigma = 1/SNR #np.max(np.abs(data))/SNR
    noise = np.random.normal(0,sigma,data.shape)
    noised_data = data + noise
    return noised_data

def get_func_bounds(function):
    f_name = function.__name__
    if f_name == "S_biX_4p":
        lower_bound = (-1,-1,1,1)
        upper_bound = (1,1,300,1000)
    elif f_name == "S_moX_2p":
        lower_bound = (-1,1)
        upper_bound = (1,1000)
    else:
        raise Exception("Not a valid function: " + f_name)

    return lower_bound, upper_bound

# Initialize Process Functions

In [62]:
def check_param_order(popt):
    #Reshaping of array to ensure that the parameter pairs all end up in the appropriate place - ensures that T22 > T21
    if (popt[-1] < popt[-2]): #We want by convention to make sure that T21 is <= T22
        for pi in range(np.size(popt)//2):
            p_hold = popt[2*pi]
            popt[2*pi] = popt[2*pi+1]
            popt[2*pi] = p_hold
    return popt

def estimate_NLLS(data, tdata, function):

    lb, ub = get_func_bounds(function)

    init_p = tuple(np.add(np.subtract(ub,lb)*np.random.uniform(0,1,np.size(lb)),lb))

    popt, pcov = curve_fit(function, tdata, data, bounds = (lb, ub), p0=init_p, max_nfev = 1500)
    
    popt = check_param_order(popt)
        
    return popt

def generate_noised_sigs(TI, tdata, SNR = SNR_fixed, params = true_params, iterations = num_iters, seed = rand_seed):
    noised_ensemble = np.zeros((iterations,np.size(tdata)))
    for iter in range(iterations):
        np.random.seed(seed+iter)
        true_sig = S_biX_6p(tdata, TI, *params)
        noised_sig = add_noise(true_sig, SNR)
        noised_ensemble[iter,:] = noised_sig
    return noised_ensemble

# Initialize Objective Functions

In [63]:
def calc_RSS_noise(TI, tdata, function, starts = num_starts):

    data = generate_noised_sigs(TI, tdata)

    iteration_RSS_values = np.zeros((data.shape[0],1))
    
    for i in range(data.shape[0]):
        noisey_signal = data[i,:]

        recreated_curves = np.zeros((starts,np.size(tdata)))
        recreated_curves_RSS = np.zeros((starts,1))
        recreated_popt = []                     #general to accomodate for different parameter sizes
        for start in range(starts):
            recreated_popt.append(estimate_NLLS(noisey_signal, tdata, function))
            recreated_curves[start,:] = function(tdata, *recreated_popt[start])
            recreated_curves_RSS[start] = np.sum((noisey_signal - recreated_curves[start,:])**2)

        iteration_RSS_values[i] = np.min(recreated_curves_RSS)

    return np.mean(iteration_RSS_values)


def calc_std(TI, tdata, function, param_select, starts = num_starts):

    data = generate_noised_sigs(TI, tdata)

    lb, ub = get_func_bounds(function)
    assert(param_select < np.size(lb))

    iteration_param_values = np.zeros((data.shape[0],np.size(lb)))
    
    for i in range(data.shape[0]):
        noisey_signal = data[i,:]

        recreated_curves = np.zeros((starts,np.size(tdata)))
        recreated_curves_RSS = np.zeros((starts,1))
        recreated_popt = []                     #general to accomodate for different parameter sizes
        for start in range(starts):
            recreated_popt.append(estimate_NLLS(noisey_signal, tdata, function))
            recreated_curves[start,:] = function(tdata, *recreated_popt[start])
            recreated_curves_RSS[start] = np.sum((noisey_signal - recreated_curves[start,:])**2)
            
        recreated_popt = np.array(recreated_popt)
        iteration_param_values[i,:] = recreated_popt[np.argmin(recreated_curves_RSS),:]

    std_values = -1*np.std(iteration_param_values, axis = 0) #have to invert to make it find a minimum

    return std_values[param_select]


# Running Optimization Functions

In [66]:
minimum_TI_moX_RSS = golden(lambda TI: calc_RSS_noise(TI, TE_array, S_moX_2p), brack = (100,300,600), tol = 10**-1)

minimum_TI_biX_std = golden(lambda TI: calc_std(TI, TE_array, S_biX_4p, 0), brack = (100,600), tol = 10**-1)

print("Golden Section Search Technique")
print("True Nullpoint 1 = {:.3f}".format(TI1star))
print("True Nullpoint 2 = {:.3f}".format(TI2star))
print("Estimated Nullpoint via RSS of moX = {:.3f}".format(minimum_TI_moX_RSS))
print("Estimated Nullpoint via std of biX = {:.3f}".format(minimum_TI_biX_std))

Golden Section Search Technique
True Nullpoint 1 = 415.888
True Nullpoint 2 = 831.777
Estimated Nullpoint via RSS of moX = 414.590
Estimated Nullpoint via std of biX = 718.034
